In [1]:
import findspark
findspark.init('/home/akshay/spark-3.5.1-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_tutorials_1").getOrCreate()

24/06/13 16:51:30 WARN Utils: Your hostname, akshay-vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/13 16:51:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/13 16:51:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = [
    (1, 'Alice', {'hair': 'blonde', 'eye': 'blue'}),
    (2, 'Bob', {'hair': 'brown', 'eye': 'brown'}),
    (3, 'Charlie', {'hair': 'black', 'eye': 'green'}),
    (4, 'David', {'hair': 'red', 'eye': 'blue'}),
    (5, 'Eve', {'hair': 'brown', 'eye': 'brown'})
]

In [3]:
schema = ["id","name","feature"]

In [4]:
df = spark.createDataFrame(data=data, schema=schema)

In [5]:
df.show(truncate=True)

+---+-------+--------------------+
| id|   name|             feature|
+---+-------+--------------------+
|  1|  Alice|{eye -> blue, hai...|
|  2|    Bob|{eye -> brown, ha...|
|  3|Charlie|{eye -> green, ha...|
|  4|  David|{eye -> blue, hai...|
|  5|    Eve|{eye -> brown, ha...|
+---+-------+--------------------+



explode

In [6]:
help(df.select())

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(pyspark.sql.pandas.map_ops.PandasMapOpsMixin, pyspark.sql.pandas.conversion.PandasConversionMixin)
 |  DataFrame(jdf: py4j.java_gateway.JavaObject, sql_ctx: Union[ForwardRef('SQLContext'), ForwardRef('SparkSession')])
 |  
 |  A distributed collection of data grouped into named columns.
 |  
 |  .. versionadded:: 1.3.0
 |  
 |  .. versionchanged:: 3.4.0
 |      Supports Spark Connect.
 |  
 |  Examples
 |  --------
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SparkSession`:
 |  
 |  >>> people = spark.createDataFrame([
 |  ...     {"deptId": 1, "age": 40, "name": "Hyukjin Kwon", "gender": "M", "salary": 50},
 |  ...     {"deptId": 1, "age": 50, "name": "Takuya Ueshin", "gender": "M", "salary": 100},
 |  ...     {"deptId": 2, "age": 60, "name": "Xinrong Meng", "gender": "F", "salary": 150},
 |  ...     {"deptId": 3, "age": 20,

In [11]:
from pyspark.sql.functions import col, explode, map_keys, map_values
df1 = df.select(col("id"), col("name"), explode(col("feature")))
df1.show(truncate=False)

+---+-------+----+------+
|id |name   |key |value |
+---+-------+----+------+
|1  |Alice  |eye |blue  |
|1  |Alice  |hair|blonde|
|2  |Bob    |eye |brown |
|2  |Bob    |hair|brown |
|3  |Charlie|eye |green |
|3  |Charlie|hair|black |
|4  |David  |eye |blue  |
|4  |David  |hair|red   |
|5  |Eve    |eye |brown |
|5  |Eve    |hair|brown |
+---+-------+----+------+



In [12]:
df1.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)



In [13]:
df2 = df.withColumn("keys", map_keys("feature"))
df2.show()

+---+-------+--------------------+-----------+
| id|   name|             feature|       keys|
+---+-------+--------------------+-----------+
|  1|  Alice|{eye -> blue, hai...|[eye, hair]|
|  2|    Bob|{eye -> brown, ha...|[eye, hair]|
|  3|Charlie|{eye -> green, ha...|[eye, hair]|
|  4|  David|{eye -> blue, hai...|[eye, hair]|
|  5|    Eve|{eye -> brown, ha...|[eye, hair]|
+---+-------+--------------------+-----------+



In [14]:
df2 = df.withColumn("keys", map_values("feature"))
df2.show()

+---+-------+--------------------+--------------+
| id|   name|             feature|          keys|
+---+-------+--------------------+--------------+
|  1|  Alice|{eye -> blue, hai...|[blue, blonde]|
|  2|    Bob|{eye -> brown, ha...|[brown, brown]|
|  3|Charlie|{eye -> green, ha...|[green, black]|
|  4|  David|{eye -> blue, hai...|   [blue, red]|
|  5|    Eve|{eye -> brown, ha...|[brown, brown]|
+---+-------+--------------------+--------------+

